In [3]:
import pandas as pd
import sklearn.metrics as metr
import numpy as np
from ignite.metrics import Precision
import torch
from torchvision import models
import os

In [86]:
m = models.resnet50(pretrained=True)
for param in m.parameters():
    param.requires_grad = False
input_num = 3
out_shape = 2
m.conv1 = torch.nn.Conv2d(input_num, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
last_layer_in_channels = m.fc.in_features
m.fc = torch.nn.Linear(last_layer_in_channels, out_shape)

In [87]:
device = 'cuda'
model_path = '../Saved_models/'
model_name = 'Saved_models'
model = m
model.load_state_dict(torch.load(model_path+model_name))
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [88]:
import os
import torchvision.transforms.functional as TF
from keras.preprocessing.image import array_to_img, img_to_array
from torch.utils.data import Dataset, DataLoader
import random
import numpy as np
import h5py


def load_image(file_name, type='image'):
    f = h5py.File(file_name, 'r')
    file_np = f['img'][()]
    if type == 'image':
        file_np = (file_np / 255.0).astype('float32')
    elif type == 'mask':
        file_np = file_np.astype('uint8')
    else:
        print('not choosed type to load')
        return
    return file_np


class MyDataset(Dataset):

    def __init__(self, train_test_id, labels_ids, args, train):

        self.train_test_id = train_test_id
        self.labels_ids = labels_ids
        self.image_path = args['image_path']
        self.attribute = args['attribute']
        self.mask_use = args['mask_use']
        self.train = train
        self.all_attributes = ['attribute_globules', 'attribute_milia_like_cyst', 'attribute_negative_network',
                               'attribute_pigment_network', 'attribute_streaks']

        self.indexes = [i for i, val in enumerate(self.all_attributes) for attr in self.attribute if attr == val]

        if train == 'valid':
            self.labels_ids = self.labels_ids[self.train_test_id['Split'] != 'train'].values.astype('uint8')
            self.train_test_id = self.train_test_id[self.train_test_id['Split'] != 'train'].ID.values
            #print('Train =', self.train, 'train_test_id.shape: ', self.train_test_id.shape)

        self.n = self.train_test_id.shape[0]

    def __len__(self):
        return self.n


    def __getitem__(self, index):

        name = self.train_test_id[index]
        path = self.image_path
        # Load image and from h5
        image = load_image(os.path.join(path, '%s.h5' % name), 'image')
        mask = np.empty([image.shape[0], image.shape[1], len(self.attribute)])
        if self.mask_use:
            for i, attr in enumerate(self.attribute):
                mask[:, :, i] = load_image(os.path.join(path, '{}_{}.h5'.format(name, attr)), 'mask')[:, :, 0]
        else:
            mask = image

        if self.mask_use:
            if self.train == 'valid':
                mask.fill(0.)
            image_with_mask = np.dstack((image, mask))
        else:
            image_with_mask = image

        labels = np.array([self.labels_ids[index, ind] for ind in self.indexes])

        return image_with_mask, labels, name


def make_loader(train_test_id, labels_ids, args, train=True, shuffle=True):

    data_set = MyDataset(train_test_id=train_test_id,
                         labels_ids=labels_ids,
                         args=args,
                         train=train)
    data_loader = DataLoader(data_set,
                             batch_size=args['batch_size'],
                             shuffle=shuffle
                             )
    return data_loader

Using TensorFlow backend.


In [81]:
class Metrics():
    
    def __init__(self):
        
        self.tp = 0
        self.fp = 0
        self.fn = 0
        self.tn = 0
        
    def update(self, y_true, y_pred):
        tn, fp, fn, tp = metr.confusion_matrix(y_true, y_pred).ravel()
        self.tp += tp
        self.fp += fp
        self.fn += fn
        self.tn += tn
        
    def compute(self):
        acc = (self.tp + self.tn) / (self.tp + self.tn + self.fp + self.fn) # TP + TN / (TP + TN + FP + FN)
        prec = self.tp / (self.tp + self.fp)                                # TP / (TP + FP)
        rec = self.tp / (self.tp + self.fn)                                 # TP / (TP + FN)
        f1 = 2 * prec * rec / (rec + prec)                                  # 2 * PREC * REC / (PREC + REC)
        return acc, prec, rec, f1
    
    def reset(self):
        self.tp = 0
        self.fp = 0
        self.fn = 0
        self.tn = 0
        

In [83]:
M = Metrics()
M.update(y.ravel(), y_pred.ravel())
print(M.compute())
M.reset()
print(M.tp, M.fp)

(0.75, 0.5, 1.0, 0.6666666666666666)
0 0


In [97]:

#### здесь как предсказывал ignite
train_test_id = pd.read_csv('../Data/train_test_id.csv')
mask_ind = pd.read_csv('../Data/mask_ind.csv')
metric = Metrics()
args = {
    'image_path': '../Data/h5/',
    'attribute':['attribute_pigment_network', 'attribute_milia_like_cyst'],
    'mask_use':False,
    'batch_size': 2
}
loader = make_loader(train_test_id, mask_ind, args, train='valid', shuffle=True)
for i, (image_batch, labels_batch, names) in enumerate(loader):
    image_batch = image_batch.permute(0, 3, 1, 2).to(device).type(torch.cuda.FloatTensor)
    labels_batch = labels_batch.to(device).type(torch.cuda.FloatTensor)
    output_probs = model(image_batch)
    outputs = torch.sigmoid(output_probs)

    print(labels_batch.data.cpu().numpy().ravel(), np.around(outputs.data.cpu().numpy().ravel()))
    
    metric.update(labels_batch.data.cpu().numpy().ravel(), np.around(outputs.data.cpu().numpy().ravel()))
    metrics = metric.compute()
    print('Acc: {:.4f} Prec: {:.4f} Recall: {:.4f}'.format(
            metrics[0], metrics[1], metrics[2]))
metric.reset()

[0. 1. 0. 1.] [0. 1. 0. 1.]
Acc: 1.0000 Prec: 1.0000 Recall: 1.0000
[0. 1. 0. 1.] [0. 1. 0. 1.]
Acc: 1.0000 Prec: 1.0000 Recall: 1.0000
[0. 1. 0. 1.] [0. 1. 0. 1.]
Acc: 1.0000 Prec: 1.0000 Recall: 1.0000
[0. 0. 0. 0.] [0. 1. 0. 1.]
Acc: 0.8750 Prec: 0.7500 Recall: 1.0000
[0. 1. 1. 1.] [0. 1. 0. 1.]
Acc: 0.8500 Prec: 0.8000 Recall: 0.8889
[0. 0. 0. 1.] [0. 1. 0. 1.]
Acc: 0.8333 Prec: 0.7500 Recall: 0.9000
[0. 0. 0. 1.] [0. 1. 0. 1.]
Acc: 0.8214 Prec: 0.7143 Recall: 0.9091
[1. 0. 0. 0.] [0. 1. 0. 1.]
Acc: 0.7500 Prec: 0.6250 Recall: 0.8333
[0. 1. 0. 1.] [0. 1. 0. 1.]
Acc: 0.7778 Prec: 0.6667 Recall: 0.8571
[0. 0. 0. 0.] [0. 1. 0. 1.]
Acc: 0.7500 Prec: 0.6000 Recall: 0.8571
[0. 1. 0. 0.] [0. 1. 0. 1.]
Acc: 0.7500 Prec: 0.5909 Recall: 0.8667
[1. 0. 0. 0.] [0. 1. 0. 1.]
Acc: 0.7083 Prec: 0.5417 Recall: 0.8125
[0. 1. 1. 1.] [0. 1. 0. 1.]
Acc: 0.7115 Prec: 0.5769 Recall: 0.7895
[0. 0. 0. 1.] [0. 1. 0. 1.]
Acc: 0.7143 Prec: 0.5714 Recall: 0.8000
[0. 0. 0. 1.] [0. 1. 0. 1.]
Acc: 0.7167 Prec: 0.

[1. 0. 0. 0.] [0. 1. 0. 1.]
Acc: 0.6789 Prec: 0.5894 Recall: 0.7178
[0. 0. 0. 1.] [0. 1. 0. 1.]
Acc: 0.6794 Prec: 0.5887 Recall: 0.7192
[0. 1. 1. 0.] [0. 1. 0. 1.]
Acc: 0.6780 Prec: 0.5880 Recall: 0.7171
[0. 1. 0. 0.] [0. 1. 0. 1.]
Acc: 0.6786 Prec: 0.5873 Recall: 0.7184
[1. 1. 0. 1.] [0. 1. 0. 1.]
Acc: 0.6791 Prec: 0.5906 Recall: 0.7177
[0. 1. 1. 0.] [0. 1. 0. 1.]
Acc: 0.6777 Prec: 0.5898 Recall: 0.7156
[0. 0. 0. 0.] [0. 1. 0. 1.]
Acc: 0.6764 Prec: 0.5853 Recall: 0.7156
[1. 0. 0. 0.] [0. 1. 0. 1.]
Acc: 0.6731 Prec: 0.5808 Recall: 0.7123
[0. 1. 0. 1.] [0. 1. 0. 1.]
Acc: 0.6756 Prec: 0.5840 Recall: 0.7150
[0. 1. 0. 1.] [0. 1. 0. 1.]
Acc: 0.6780 Prec: 0.5871 Recall: 0.7176
[0. 1. 1. 1.] [0. 1. 0. 1.]
Acc: 0.6786 Prec: 0.5902 Recall: 0.7169
[0. 1. 0. 0.] [0. 1. 0. 1.]
Acc: 0.6791 Prec: 0.5896 Recall: 0.7182
[0. 0. 0. 1.] [0. 1. 0. 1.]
Acc: 0.6796 Prec: 0.5889 Recall: 0.7195
[0. 0. 0. 1.] [0. 1. 0. 1.]
Acc: 0.6801 Prec: 0.5882 Recall: 0.7207
[0. 1. 0. 0.] [0. 1. 0. 1.]
Acc: 0.6807 Prec: 0.

# Data

In [13]:
path = '../Data/'
tti_name = 'train_test_id.csv'
mi_name = 'mask_ind.csv'
os.listdir('../Data/')

['h5', 'train_test_id.csv', 'skin_images', 'mask_ind.csv', 'skin_masks']

In [11]:
tti = pd.read_csv(path+tti_name)
tti.head()

,ID,Split
0,ISIC_0000000,train
1,ISIC_0000001,train
2,ISIC_0000003,train
3,ISIC_0000004,train
4,ISIC_0000006,train


In [14]:
mi = pd.read_csv(path+mi_name)
mi.head()

,globules,milia_like_cyst,negative_network,pigment_network,streaks
0,0,0,0,1,0
1,1,0,0,1,1
2,0,1,0,1,0
3,0,0,0,0,0
4,0,0,0,1,0


# Test step

In [33]:
import os
import pandas as pd
import sys
sys.path.insert(0, '../')
from train import make_step
from models import create_model
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.backends import cudnn
import torch
from metrics import Metrics
from loss import LossBinary

In [52]:
class Args:
    
    def __init__(self):
        self.attribute = ['attribute_milia_like_cyst', 'attribute_pigment_network']
        self.pretrained = True
        self.jaccard_weight = 0.
        self.show_model = True
        self.mask_use = True
        self.freezing = True
        self.prob = 0.1
        self.image_path = '../Data/h5/'
        self.batch_size = 2
        self.model = 'resnet50'
        self.lr = 0.00001
        self.augment_list = []
        self.N = 0
device = 'cuda'

In [53]:
args = Args()
train_test_id = pd.read_csv('../Data/train_test_id.csv')
mask_ind = pd.read_csv('../Data/mask_ind.csv')

In [54]:
cudnn.benchmark = True
torch.set_default_tensor_type(torch.cuda.FloatTensor)
model, optimizer = create_model(args, device)
metric = Metrics()
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.8, patience=10, verbose=True)
criterion = LossBinary(args.jaccard_weight)
metric = Metrics()
print(model)

ResNet(
  (conv1): Conv2d(5, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [55]:
mode = 'train'
ep = 0
results = pd.DataFrame(columns=['mask_use', 'freeze_mode', 'lr', 'exp', 'train_mode', 'epoch', 'loss', 'prec',
                                    'recall'])
metrics, results = make_step(model=model, mode=mode, train_test_id=train_test_id, mask_ind=mask_ind,
                                                args=args, device=device, criterion=criterion, optimizer=optimizer,
                                                results=results, metric=metric, epoch=ep, scheduler=scheduler)

Epoch: 0 Loss: 0.168882 Accuracy: 0.8636 Precision: 0.8715 Recall: 0.8940 F1: 0.8826 Time: 113.1284


In [56]:
mode = 'valid'
ep = 0
results = pd.DataFrame(columns=['mask_use', 'freeze_mode', 'lr', 'exp', 'train_mode', 'epoch', 'loss', 'prec',
                                    'recall'])
metrics, results = make_step(model=model, mode=mode, train_test_id=train_test_id, mask_ind=mask_ind,
                                                args=args, device=device, criterion=criterion, optimizer=optimizer,
                                                results=results, metric=metric, epoch=ep, scheduler=scheduler)

Epoch: 0 Loss: 0.683116 Accuracy: 0.6617 Precision: 0.7229 Recall: 0.6838 F1: 0.7028 Time: 5.5147
